## 8.DBSCAN
[Теория Хабр](https://habr.com/ru/articles/322034/)

[Теория sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)

[Датасет](https://www.kaggle.com/datasets/ankit8467/dataset-for-dbscan)
